# Pre-analysis progress checker notebook

Check how many experiments & positions have 

- alignment 
- segmentation (and if complete) 
- tracks

In [2]:
import os
import re
import glob
import pandas as pd
from natsort import natsorted
pd.set_option('display.max_rows', 150) 
# def sort(list_):
#     list_ = sorted(list_, 
#     key = lambda x: [int(y) for y in re.findall(r'\d+', x)])
#     return list_
    

In [3]:
root_dir = '/home/nathan/data/kraken/ras/'

In [4]:
expt_list = sorted([expt for expt in os.listdir(root_dir) 
                    if 'ND' in expt and os.path.isdir(os.path.join(root_dir, expt))], 
                    key = lambda x: [int(y) for y in re.findall(r'\d+', x)])
print(expt_list)

['ND0013', 'ND0014', 'ND0016', 'ND0017', 'ND0018', 'ND0019', 'ND0020', 'ND0021']


In [5]:
progress = []
for expt in expt_list:
    ### sorted position list
    pos_list = sorted([pos for pos in os.listdir(os.path.join(root_dir, expt)) 
                       if 'Pos' in pos 
                       and os.path.isdir(os.path.join(root_dir, expt, pos))],
                       key = lambda x: [int(y) for y in re.findall(r'\d+', x)])
    for pos in pos_list:
        ### does any alignment tensors exist?
        directory = os.path.join(root_dir, expt, pos)
        directory_files = os.listdir(directory)
        tensor_file = len([fn for fn in directory_files if 'reversed_clipped_transform_tensor' in fn and '.npy' in fn])

        ## do masks exist
        image_dir = os.path.join(directory, f'{pos}_images')
        n_masks = len(glob.glob(f'{image_dir}/*channel099*.tif'))
        n_imgs = len(glob.glob(f'{image_dir}/*channel001*.tif'))
        
        if n_imgs == 0:
            image_files = 0
        else:
            image_files = int((n_masks/n_imgs)*100)

        ## do objects exist
        obj_files = len([fn for fn in directory_files if 'obj' in fn and '.h5' in fn])
        
        ## do tracks exist 
        track_files = len([fn for fn in directory_files if 'tracks' in fn and '.h5' in fn])
        
        ## append data
        progress.append((expt, pos, tensor_file, image_files, obj_files, track_files))
### turn into df
progress_df = pd.DataFrame(progress, columns=['Experiment', 'Position', 'Transformations', 'Masks (% complete)', 'Object files','Tracks'])

In [8]:
progress_df

,Experiment,Position,Transformations,Masks (% complete),Object files,Tracks
0,ND0013,Pos0,1,100,4,0
1,ND0013,Pos1,1,100,4,0
2,ND0013,Pos2,1,100,4,0
3,ND0013,Pos3,1,100,4,0
4,ND0013,Pos4,1,100,4,0
5,ND0013,Pos5,1,100,4,0
6,ND0013,Pos6,1,100,4,0
7,ND0013,Pos7,1,100,4,0
8,ND0013,Pos8,1,100,4,0
9,ND0013,Pos9,1,100,4,0


In [9]:
progress_df.loc[progress_df['Masks (% complete)'] != 100.0]

,Experiment,Position,Transformations,Masks (% complete),Object files,Tracks
13,ND0013,Pos13,1,60,4,0
88,ND0020,Pos4,0,57,0,0
89,ND0020,Pos5,0,0,0,0
90,ND0020,Pos6,0,0,0,0
91,ND0020,Pos7,0,0,0,0
92,ND0020,Pos8,0,0,0,0
93,ND0020,Pos9,0,0,0,0
94,ND0020,Pos10,0,0,0,0
95,ND0020,Pos11,0,0,0,0
96,ND0020,Pos12,0,0,0,0
